In [2]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
from numpy import linspace, array, logspace, sin, cos, pi, arange, sqrt, arctan2, arccos


import astropy.io.fits as pyfits



In [71]:
#Grab the existing table columns we want 
cat = Table.read('../3D_CMZ/LBV_plots_stuff/CMZ_cloud_catalogue_data.tex')
absorp_tab = Table.read('../3D_CMZ/LBV_plots_stuff/absorption_table_merged.tex')

absorp_NF_tab = pd.read_csv('../3D_CMZ/LBV_plots_stuff/paper3_agreement_catalogue.txt',sep=',',
                            usecols=[0,1,2,3,4], names=['i', 'l', 'b', 'v', 'NF'],
                            header=None, index_col=False)


cat_index = cat['index']
cloud_list = cat['cloud_name']
l = cat['l']
b = cat['b']
v = cat['v']
sigma = cat['sigma']
rad = cat['rad']
NF_decision = cat['NF_decision']

lb_pixel_mask = cat['lb_pixel_mask']

corr_coeff = cat['r_p_max']
flux_diff = cat['flux_diff_max']
flux_ratio = cat['flux_ratio_max']

absorp_value = absorp_tab['fraction_value_peak_mask']
absorp_index = absorp_tab['leaf_id']

In [86]:
absorp_NF_tab['NF'][np.where(absorp_NF_tab['NF']=='Far')[0]] = 'F'
absorp_NF_tab['NF'][np.where(absorp_NF_tab['NF']=='Near')[0]] = 'N'

NF_absorp = absorp_NF_tab['NF'].values

/var/folders/4b/_cshhy012yj8k_187jxt881r0000gn/T/ipykernel_70390/3162031848.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  absorp_NF_tab['NF'][np.where(absorp_NF_tab['NF']=='Far')[0]] = 'F'
/var/folders/4b/_cshhy012yj8k_187jxt881r0000gn/T/ipykernel_70390/3162031848.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  absorp_NF_tab['NF'][np.where(absorp_NF_tab['NF']=='Near')[0]] = 'N'


In [90]:
###just need this file for the multi-comp clouds
large_cat = Table.read('../3D_CMZ/LBV_plots_stuff/CMZ_cloud_catalogue_data.csv')
large_cat_index = large_cat['index']
large_cat_v = large_cat['v']

In [91]:
### For the stddevs, we need to find them for both the single 
### and multi-component clouds separately 
### then combine into the table 

#Find single v component clouds
single_v = {x for x in list(cloud_list) if list(cloud_list).count(x) == 1}


sing_index =[]
for i in range(len(cloud_list)):
    if cloud_list[i] in single_v:
        sing_index.append(i)
        

#find indexes with multiple velocities 
dup = {x for x in list(cloud_list) if list(cloud_list).count(x) > 1}

dup_index =[]
for i in range(len(cloud_list)):
    if cloud_list[i] in dup:
        dup_index.append(i)
        
id_v_list = []
for i in range(len(large_cat_index)):
    id_v_list.append('{}_{}'.format(large_cat_index[i], large_cat_v[i]))

In [92]:
flux_ratio_std = []
flux_diff_std = []

for i in range(len(cat_index)):
    if cat_index[i] in cat_index[sing_index]:
        cloud = cloud_list[i]
        
        cloud_flux_diff = pyfits.open('../3D_CMZ/Cloud_masks/{}/{}_flux_diff.fits'.format(cloud, cloud))[0].data
        fdiff_std = np.abs(np.nanstd(cloud_flux_diff))
        flux_diff_std.append('{:.2f}'.format(np.round(fdiff_std,2)))

        cloud_flux_ratio = pyfits.open('../3D_CMZ/Cloud_masks/{}/{}_flux_ratio.fits'.format(cloud, cloud))[0].data
        fratio_std = np.abs(np.nanstd(cloud_flux_ratio))
        flux_ratio_std.append('{:.2f}'.format(np.round(fratio_std,2)))
        
    if cat_index[i] in cat_index[dup_index]:
        cloud_flux_diff = pyfits.open('../3D_CMZ/Sub_masks/{}/max_submask/{}_{}_flux_diff_maxcutout_isolated.fits'.format(large_cat_index[i], large_cat_index[i],large_cat_v[i] ))[0].data
        fdiff_std = np.abs(np.nanstd(cloud_flux_diff))
        flux_diff_std.append('{:.2f}'.format(np.round(fdiff_std,2)))

        cloud_flux_ratio = pyfits.open('../3D_CMZ/Sub_masks/{}/max_submask/{}_{}_flux_ratio_maxcutout_isolated.fits'.format(large_cat_index[i], large_cat_index[i],large_cat_v[i] ))[0].data
        fratio_std = np.abs(np.nanstd(cloud_flux_ratio))
        flux_ratio_std.append('{:.2f}'.format(np.round(fratio_std,2)))

/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [102]:
### absorption table only has the ids with measurement available

#create nan array with size equal to 'index'
absorp_arr = []
absorp_NF = []

#fill in where indices match. Make nan where no match.
for i in cat_index:
    if i in absorp_index:
        if i =='16a': #16a is an empty mask. replace with nan
            absorp_arr.append(np.nan)
        else: 
            absorp_arr.append(absorp_value[np.where(absorp_index==str(i))[0][0]])
    else: 
        absorp_arr.append(np.nan)
        
for i in cat_index:
    if i in absorp_NF_tab['i'].values:
        if i =='16a': #16a is an empty mask. replace with nan
            absorp_NF.append(np.nan)
        else: 
            absorp_NF.append(absorp_NF_tab['NF'][np.where(absorp_NF_tab['i']==str(i))[0][0]])
    else: 
        absorp_NF.append(np.nan)


In [104]:
### Construct new table 

synth_table = Table()
synth_table['leaf_id'] = cat_index
synth_table['cloud_name'] = cat['cloud_name']
synth_table['l'] = l
synth_table['b'] = b
synth_table['v'] = v
synth_table['sigma'] = sigma
synth_table['rad'] = rad
synth_table['corr_coeff'] = corr_coeff.round(2)
synth_table['flux_diff'] = flux_diff.round(2)
synth_table['flux_diff_stdv'] = flux_diff_std
synth_table['flux_ratio'] = flux_ratio.round(2)
synth_table['flux_ratio_stdv'] = flux_ratio_std
synth_table['absorp_value'] = absorp_arr
synth_table['NF_decision'] = NF_decision
synth_table['absorp_NF'] = absorp_NF
synth_table['lb_pixel_mask'] = lb_pixel_mask

synth_table


leaf_id,cloud_name,l,b,v,sigma,rad,corr_coeff,flux_diff,flux_diff_stdv,flux_ratio,flux_ratio_stdv,absorp_value,NF_decision,absorp_NF,lb_pixel_mask
str3,str14,float64,float64,int64,int64,float64,float64,float64,str6,float64,str4,float64,str3,str32,int64
1,G359.475-0.044,-0.525,-0.044,-102,6,4.3,0.26,31.67,59.09,0.51,0.20,nan,LN,nan,1
2,G359.508-0.135,-0.492,-0.135,-56,9,8.7,0.38,-20.61,56.22,0.62,0.19,nan,N,nan,1
3,G359.561-0.001,-0.439,-0.001,-90,8,2.7,0.22,-6.67,23.73,0.55,0.09,2.15,U,F,1
4a,G359.595-0.223,-0.405,-0.223,-27,13,4.6,0.43,-26.88,73.95,0.57,0.30,nan,N,nan,1
4b,G359.595-0.223,-0.405,-0.223,-20,8,4.6,0.26,-25.56,101.97,0.56,0.42,nan,LN,nan,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29a,G1.075-0.049,1.075,-0.049,74,6,8.5,-0.23,-111.22,130.85,0.87,1.17,nan,nan,nan,0
29b,G1.075-0.049,1.075,-0.049,85,16,8.5,-0.01,-63.43,100.33,0.33,0.91,nan,nan,nan,1
30a,G1.601+0.012,1.601,0.012,48,5,3.8,0.34,-65.28,2.81,-0.01,0.04,nan,nan,nan,1


In [105]:
synth_table.write("/Users/danilipman/Documents/Research/UConn/CMZ_SYNTH/synth_table.tex", overwrite=True)

    


In [106]:
print(absorp_index[22], absorp_value[22])

17c 1.63


In [111]:
synth_table[16]

leaf_id,cloud_name,l,b,v,sigma,rad,corr_coeff,flux_diff,flux_diff_stdv,flux_ratio,flux_ratio_stdv,absorp_value,NF_decision,absorp_NF,lb_pixel_mask
str3,str14,float64,float64,int64,int64,float64,float64,float64,str6,float64,str4,float64,str3,str32,int64
11b,G0.014-0.016,0.014,-0.016,45,9,3.4,-0.35,-125.23,101.24,1.0,0.19,2.8,F,nan,0
